In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import time

from PIL import Image
import cv2
import imageio
import os
import natsort
import matplotlib.pyplot as plt

from rawdata_preprocessing import create_graph

from collections import defaultdict
from datetime import datetime, timedelta

In [18]:
G, df = create_graph("otc")
G.edges()

OutEdgeView([(1, 15), (1, 5), (1, 32), (1, 34), (1, 17), (1, 31), (1, 13), (1, 20), (1, 54), (1, 56), (1, 71), (1, 74), (1, 78), (1, 68), (1, 81), (1, 101), (1, 119), (1, 2), (1, 76), (1, 23), (1, 138), (1, 110), (1, 145), (1, 114), (1, 111), (1, 7), (1, 165), (1, 19), (1, 223), (1, 225), (1, 132), (1, 227), (1, 104), (1, 64), (1, 245), (1, 247), (1, 234), (1, 251), (1, 260), (1, 261), (1, 109), (1, 271), (1, 274), (1, 141), (1, 296), (1, 180), (1, 311), (1, 312), (1, 139), (1, 324), (1, 356), (1, 51), (1, 384), (1, 399), (1, 438), (1, 442), (1, 144), (1, 219), (1, 47), (1, 320), (1, 464), (1, 431), (1, 486), (1, 416), (1, 505), (1, 421), (1, 80), (1, 29), (1, 511), (1, 494), (1, 534), (1, 537), (1, 563), (1, 567), (1, 569), (1, 215), (1, 579), (1, 61), (1, 266), (1, 21), (1, 630), (1, 677), (1, 695), (1, 672), (1, 773), (1, 647), (1, 396), (1, 353), (1, 9), (1, 779), (1, 545), (1, 1134), (1, 3), (1, 8), (1, 197), (1, 60), (1, 41), (1, 39), (1, 33), (1, 6), (1, 10), (1, 270), (1, 539),

In [19]:
df

,source,target,rating,time
0,6,2,4,101108
1,6,5,2,101108
2,1,15,1,101108
3,4,3,7,101108
4,13,16,8,101108
...,...,...,...,...
35587,4499,1810,1,160124
35588,2731,3901,5,160124
35589,2731,4897,5,160124
35590,13,1128,1,160124


In [20]:
transaction_src_dict = defaultdict(list)
transaction_time_dict = defaultdict(list)
for i in range(len(df)):
    src, trg, rating, time = df.iloc[i]
    
    str_year = time[:2]
    str_month = time[2:4]
    str_day = time[4:]
    
    year = int("20" + str_year)
    month = int(str_month)
    day = int(str_day)
    
    date_time_form = datetime(year, month, day)
    
    transaction_src_dict[src].append([trg, rating, date_time_form])
    transaction_time_dict[date_time_form].append([src,trg,rating])

In [21]:
transaction_time_dict

defaultdict(list,
            {datetime.datetime(2010, 11, 8, 0, 0): [[6, 2, 4],
              [6, 5, 2],
              [1, 15, 1],
              [4, 3, 7],
              [13, 16, 8],
              [13, 10, 8]],
             datetime.datetime(2010, 11, 10, 0, 0): [[7, 5, 1],
              [2, 21, 5],
              [2, 20, 5],
              [21, 2, 5]],
             datetime.datetime(2010, 11, 11, 0, 0): [[21, 1, 8],
              [21, 10, 8],
              [21, 8, 9],
              [21, 3, 7],
              [17, 3, 5],
              [17, 23, 1]],
             datetime.datetime(2010, 11, 12, 0, 0): [[10, 1, 8],
              [10, 6, 7],
              [10, 21, 8],
              [10, 8, 1],
              [10, 25, 10],
              [10, 2, 7],
              [10, 3, 7]],
             datetime.datetime(2010, 11, 13, 0, 0): [[4, 26, 1],
              [26, 4, 1],
              [5, 1, 3],
              [5, 6, 3],
              [5, 7, 1]],
             datetime.datetime(2010, 11, 14, 0, 0): [[1

In [22]:
time_decay_dict = defaultdict(list)
for date_time, transactions in transaction_time_dict.items():
    for trans in transactions:
        time_decay_dict[trans[0]].append([trans[1], trans[2], date_time])

In [23]:
time_decay_dict

defaultdict(list,
            {6: [[2, 4, datetime.datetime(2010, 11, 8, 0, 0)],
              [5, 2, datetime.datetime(2010, 11, 8, 0, 0)],
              [4, 2, datetime.datetime(2010, 11, 14, 0, 0)],
              [7, 5, datetime.datetime(2010, 11, 27, 0, 0)],
              [114, 2, datetime.datetime(2011, 1, 29, 0, 0)],
              [32, 1, datetime.datetime(2011, 1, 29, 0, 0)],
              [173, 3, datetime.datetime(2011, 3, 6, 0, 0)],
              [258, 1, datetime.datetime(2011, 4, 9, 0, 0)],
              [268, 3, datetime.datetime(2011, 4, 12, 0, 0)],
              [219, 4, datetime.datetime(2011, 4, 30, 0, 0)],
              [198, 2, datetime.datetime(2011, 5, 8, 0, 0)],
              [35, 4, datetime.datetime(2011, 7, 22, 0, 0)],
              [664, 1, datetime.datetime(2011, 7, 22, 0, 0)],
              [937, 1, datetime.datetime(2011, 7, 23, 0, 0)],
              [384, 6, datetime.datetime(2011, 7, 29, 0, 0)],
              [521, 1, datetime.datetime(2011, 8, 3, 0, 0)],

In [24]:
# def scale_dif_days(date1, date2, days):

#     dif_days = np.abs((date2 - date1).days)
#     mok = dif_days // days
# #     print(date1,"  ", date2,"  " ,mok)
#     return (1/2) ** mok

In [25]:
def cal_dif_days(date1, date2, days):
    dif_days = np.abs((date2 - date1).days)
    mok = dif_days // days
    return mok

In [26]:
my_result = list()

for k, transactions in time_decay_dict.items():
    k_th_trans_list = transactions.copy()
    
    for i in range(len(k_th_trans_list)):
        semi_result = defaultdict(list)
        for j in range(i+1):
            if i == 0:
                my_result.append((k, k_th_trans_list[i][0], k_th_trans_list[i][1]))
                continue
            if i == j:
                continue
            
            idx = cal_dif_days(k_th_trans_list[i][2], k_th_trans_list[j][2], 30)
            semi_result[idx].append(k_th_trans_list[j][1])
            
        if len(semi_result) != 0:
            c_sum = 0
            avg_sum = 0
            for sc, ratings in semi_result.items():
                c = (1/2)**sc
                c_sum += c
                avg_sum += np.mean(ratings) * c
            mu = avg_sum / c_sum
            r_uv = ((k_th_trans_list[i][1] - mu) / 4) + k_th_trans_list[i][1]
            my_result.append((k, k_th_trans_list[i][0], r_uv))

In [27]:
len(my_result)

35592

In [28]:
my_result

[(6, 2, 4),
 (6, 5, 1.5),
 (6, 4, 1.75),
 (6, 7, 5.583333333333333),
 (6, 114, 1.6875),
 (6, 32, 0.6875),
 (6, 173, 3.2875),
 (6, 258, 0.6018518518518519),
 (6, 268, 3.3177966101694913),
 (6, 219, 4.428066037735849),
 (6, 198, 1.838827838827839),
 (6, 35, 4.331978319783198),
 (6, 664, 0.3577396657871593),
 (6, 937, 0.61103781882146),
 (6, 384, 6.975480339588919),
 (6, 521, 0.5550799086757991),
 (6, 280, 0.5113168724279836),
 (6, 687, 0.9045036650515592),
 (6, 1386, 3.390542161070437),
 (6, 537, 5.599796168582375),
 (6, 1317, 1.665618542808373),
 (6, 1566, 0.5382789806246818),
 (6, 2034, 0.6900760605060102),
 (6, 2455, 0.9944639768663083),
 (6, 1383, -12.751768024274856),
 (6, 1810, 5.067320037810991),
 (6, 2028, 1.1301188690765473),
 (6, 1363, 10.053919048454008),
 (6, 10, 3.4563186896990876),
 (6, 1, 9.749768734543453),
 (6, 1018, 0.3612673826197276),
 (6, 1832, 0.4797203079225708),
 (6, 1624, 3.0503404880266016),
 (6, 550, 0.8411584576106356),
 (6, 1331, -12.884605706429408),
 (6, 26

In [29]:
for u, v, nr in my_result:
    G[u][v]["normrating"] = float(nr)

In [30]:
nx.write_gml(G, "../gae_Bitcoin/data/newdata/bitcoinotc_compre.gml")

In [17]:
rating_dict = nx.get_edge_attributes(G, "normrating")
rating_dict

{(2, 402): 1.0,
 (2, 1127): 1.0,
 (2, 168): 11.0,
 (2, 10): 2.8333333333333335,
 (2, 1632): 1.625,
 (2, 710): 1.7916666666666665,
 (2, 512): 0.65,
 (2, 917): 1.9833333333333334,
 (2, 436): 0.7083333333333333,
 (2, 1209): 2.123809523809524,
 (2, 54): 9.552380952380952,
 (2, 74): 6.742857142857143,
 (2, 379): 0.43055555555555547,
 (2, 2698): 0.44166666666666665,
 (2, 769): 1.7783333333333333,
 (2, 446): 1.8138888888888887,
 (2, 4): 10.612063492063491,
 (2, 312): 6.78,
 (2, 986): 0.11559139784946226,
 (2, 1603): 0.43817204301075263,
 (2, 469): 8.045698924731182,
 (2, 148): 0.40591397849462363,
 (2, 44): 0.5637992831541219,
 (2, 45): 3.0505376344086024,
 (2, 200): 0.5333333333333333,
 (2, 2446): 0.5579109062980031,
 (2, 503): 1.8287037037037037,
 (2, 987): 0.5787037037037036,
 (2, 504): 0.5914021164021164,
 (2, 7401): 0.6017917267917268,
 (2, 532): 0.6104497354497354,
 (2, 1922): 0.6066646316646316,
 (2, 237): 0.612944066515495,
 (2, 209): 0.6183862433862433,
 (2, 420): 0.5781746031746031,

In [18]:
sorted(rating_dict.items(), key=lambda x:x[1], reverse=True)

[((7598, 7604), 15.0),
 ((7536, 7), 15.0),
 ((7517, 62), 15.0),
 ((5533, 811), 15.0),
 ((148, 2), 14.99982658325625),
 ((218, 261), 14.981229111265918),
 ((7531, 7521), 14.957692307692309),
 ((7598, 1691), 14.537037037037036),
 ((269, 60), 13.623840897999024),
 ((7398, 2408), 13.590579710144928),
 ((7565, 5533), 13.369567704502737),
 ((7536, 62), 13.333333333333334),
 ((7517, 7), 13.333333333333334),
 ((7336, 3), 13.333333333333334),
 ((1691, 185), 13.242670953993683),
 ((7565, 7536), 13.112548593843028),
 ((7536, 7517), 13.0),
 ((104, 15), 12.99561403508772),
 ((30, 7), 12.959165580536549),
 ((42, 2), 12.81875),
 ((354, 224), 12.749994277997757),
 ((9, 20), 12.74490857460461),
 ((7562, 41), 12.661116512720792),
 ((181, 21), 12.59090909090909),
 ((603, 5), 12.583333333333334),
 ((286, 170), 12.55598911725868),
 ((7595, 123), 12.545625546806649),
 ((2, 37), 12.539760323864519),
 ((97, 177), 12.500031692834737),
 ((7598, 7600), 12.5),
 ((7536, 5533), 12.5),
 ((5533, 7565), 12.5),
 ((26, 

In [23]:
G1 = nx.read_gml("../gae_Bitcoin/data/bitcoinalpha.gml")

In [24]:
G1.nodes()

NodeView(('1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157',

In [29]:
G2 = nx.read_gml("../gae_Bitcoin/data/bitcoinalphanorm2.gml", destringizer=int)
G2.edges()

OutEdgeView([(2, 402), (2, 1127), (2, 168), (2, 10), (2, 1632), (2, 710), (2, 512), (2, 917), (2, 436), (2, 1209), (2, 54), (2, 74), (2, 379), (2, 2698), (2, 769), (2, 446), (2, 4), (2, 312), (2, 986), (2, 1603), (2, 469), (2, 148), (2, 44), (2, 45), (2, 200), (2, 2446), (2, 503), (2, 987), (2, 504), (2, 7401), (2, 532), (2, 1922), (2, 237), (2, 209), (2, 420), (2, 774), (2, 564), (2, 2459), (2, 1409), (2, 1222), (2, 1090), (2, 238), (2, 355), (2, 382), (2, 353), (2, 1936), (2, 2488), (2, 109), (2, 658), (2, 1225), (2, 1232), (2, 299), (2, 1421), (2, 1098), (2, 251), (2, 99), (2, 2497), (2, 2499), (2, 508), (2, 263), (2, 65), (2, 1635), (2, 63), (2, 1957), (2, 915), (2, 777), (2, 119), (2, 236), (2, 7400), (2, 1977), (2, 182), (2, 1929), (2, 7508), (2, 217), (2, 1998), (2, 473), (2, 42), (2, 449), (2, 2006), (2, 35), (2, 1693), (2, 471), (2, 271), (2, 250), (2, 113), (2, 91), (2, 40), (2, 23), (2, 767), (2, 150), (2, 424), (2, 140), (2, 39), (2, 672), (2, 1709), (2, 862), (2, 272), (2,

In [ ]:
d